# Configurare baza de date

In [399]:
!pip install neo4j pandas
from neo4j import GraphDatabase
import pandas as pd
# Importăm configurarea privată
from config import URI, USER, PASSWORD

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

def run_query(query, parameters=None):
    """Funcție helper pentru rularea query-urilor"""
    with driver.session() as session:
        result = session.run(query, parameters or {})
        return [record.data() for record in result]

try:
    driver.verify_connectivity()
    print("✓ Conectare reușită la Neo4j!")
except Exception as e:
    print(f"✗ Eroare la conectare: {e}")

✓ Conectare reușită la Neo4j!


## Curatare baza de date

In [400]:
clear_query = "MATCH (n) DETACH DELETE n"
run_query(clear_query)
print("Baza de date curățată!")

Baza de date curățată!


# Schema bazei de date

### 1. Noduri (Node Types)

| Label | Proprietăți | Descriere |
| :--- | :--- | :--- |
| **User** | `id`, `name`, `role`, `languages`, `soft_skills`, `open_to_work` | Nodul central (Studenți, Angajați, Recruiteri) |
| **Company** | `name`, `industry`, `work_culture`, `city` | Organizații și companii angajatoare |
| **University** | `name`, `country`, `rank` | Instituții de învățământ (pentru rețeaua Alumni) |
| **Project** | `name`, `tech_stack`, `type` | Proiecte (Open Source, Hackathon, Personale) |
| **Certification** | `name`, `issuer`, `difficulty` | Acreditări profesionale (ex: AWS, PMP) |
| **Post** | `id`, `topics`, `timestamp` | Conținut social (articole, știri) pentru interese |

### 2. Relații (Relationship Types)

| Tip Relație | De la -> Către | Proprietăți Cheie | Descriere |
| :--- | :--- | :--- | :--- |
| **WORKS_AT** | `User` -> `Company` | `role`, `current`, `end_year` | Istoricul profesional sau jobul curent |
| **STUDIED_AT** | `User` -> `University` | `degree`, `year` | Conectează userul de Alma Mater |
| **CONTRIBUTED_TO**| `User` -> `Project` | `commits` | Arată implicarea tehnică într-un proiect |
| **EARNED** | `User` -> `Certification`| `year` | Validează competențele userului |
| **FOLLOWS** | `User` -> `User` | *(fără proprietăți)* | Relația socială directă (prietenie/follow) |
| **ENGAGED_WITH** | `User` -> `Post` | `type` (Like, Comment) | Interes implicit (fără follow direct) |

### 3. Constrângeri (Data Integrity Constraints)

| Tip Constrângere | Target (Nod.proprietate) | Descriere                                                                                               |
| :--- | :--- |:--------------------------------------------------------------------------------------------------------|
| **UNIQUENESS** | `User.id` | Asigură că fiecare utilizator are un identificator unic în sistem. |
| **UNIQUENESS** | `Company.name` | Garantează că o companie apare o singură dată în graf (previne duplicate ex: "Google" vs "google").     |
| **UNIQUENESS** | `University.name` | Asigură consistența rețelei de Alumni; toți absolvenții sunt legați de același nod unic.                |
| **UNIQUENESS** | `Post.id` | Identificator unic pentru conținut, necesar pentru a gestiona like-uri și comentarii corect.            |
| **EXISTENCE** | `User.name` | (Opțional) Impune ca orice nod creat cu eticheta `User` să aibă obligatoriu un nume completat.          |

### Constrangeri

In [401]:
constrangeri_queries = [
    "CREATE CONSTRAINT user_id_unique IF NOT EXISTS FOR (u:User) REQUIRE u.id IS UNIQUE",
    "CREATE CONSTRAINT company_name_unique IF NOT EXISTS FOR (c:Company) REQUIRE c.name IS UNIQUE",
    "CREATE CONSTRAINT uni_name_unique IF NOT EXISTS FOR (u:University) REQUIRE u.name IS UNIQUE",
    "CREATE CONSTRAINT post_id_unique IF NOT EXISTS FOR (p:Post) REQUIRE p.id IS UNIQUE",
    "CREATE CONSTRAINT project_name_unique IF NOT EXISTS FOR (p:Project) REQUIRE p.name IS UNIQUE",
    "CREATE CONSTRAINT cert_name_unique IF NOT EXISTS FOR (c:Certification) REQUIRE c.name IS UNIQUE"
]

for c in constrangeri_queries:
    try:
        run_query(c)
    except Exception as e:
        print(f"Constrângere deja existentă sau eroare: {e}")

print("Constrângeri și indecsi create!")

Constrângeri și indecsi create!


### Noduri

In [402]:
#COMPANII
companii = """
CREATE
  (c1:Company {name: 'Google', industry: 'Tech', work_culture: ['Innovative', 'Scale'], city: 'Bucharest'}),
  (c2:Company {name: 'UiPath', industry: 'Automation', work_culture: ['Fast-paced', 'Remote'], city: 'Bucharest'}),
  (c3:Company {name: 'Bitdefender', industry: 'Security', work_culture: ['Security', 'Engineering'], city: 'Cluj-Napoca'}),
  (c4:Company {name: 'BCR', industry: 'Finance', work_culture: ['Corporate', 'Stable'], city: 'Bucharest'}),
  (c5:Company {name: 'Revolut', industry: 'FinTech', work_culture: ['Hustle', 'Data'], city: 'Remote'})
"""
run_query(companii)
print("Companii create!")

Companii create!


In [403]:
#UNIVERSITĂȚI
univ = """
CREATE
  (uni1:University {name: 'Politehnica Bucuresti', country: 'RO', rank: 800}),
  (uni2:University {name: 'Universitatea Bucuresti', country: 'RO', rank: 900}),
  (uni3:University {name: 'UBB Cluj', country: 'RO', rank: 850}),
  (uni4:University {name: 'ASE', country: 'RO', rank: 1000})
"""
run_query(univ)
print("Universități create!")


Universități create!


In [404]:
#CERTIFICARI
certificari = """
CREATE
  (cert1:Certification {name: 'AWS Solutions Architect', issuer: 'Amazon', difficulty: 5}),
  (cert2:Certification {name: 'CKA (Kubernetes)', issuer: 'CNCF', difficulty: 5}),
  (cert3:Certification {name: 'PMP', issuer: 'PMI', difficulty: 4})
"""
run_query(certificari)
print("Certificări create!")

Certificări create!


In [405]:
#PROIECTE & POSTĂRI
proiectePostari = """
CREATE
  (p1:Project {name: 'OpenVision AI', tech_stack: ['Python', 'PyTorch'], type: 'Open Source'}),
  (p2:Project {name: 'LearnGraph', tech_stack: ['Neo4j', 'Java'], type: 'Hackathon'}),
  (post1:Post {id: 'p001', topics: ['AI', 'Future'], timestamp: datetime({year:2024, month:3, day:10})}),
  (post2:Post {id: 'p002', topics: ['Hiring', 'Java'], timestamp: datetime({year:2024, month:3, day:12})})
"""
run_query(proiectePostari)
print("Proiecte și Postări create!")

Proiecte și Postări create!


In [406]:
#USERS
useri = """
CREATE
  (u1:User {id: 'u1', name: 'Ana Popa', role: 'Staff Engineer', languages: ['en', 'ro'], soft_skills: ['Leadership'], open_to_work: false}),
  (u2:User {id: 'u2', name: 'Bogdan Ionescu', role: 'DevOps Lead', languages: ['en', 'fr'], soft_skills: ['Problem Solving'], open_to_work: true}),
  (u3:User {id: 'u3', name: 'Cristi Stan', role: 'Security Expert', languages: ['en', 'de'], soft_skills: ['Audit'], open_to_work: false}),
  (u4:User {id: 'u4', name: 'David Munteanu', role: 'Frontend Dev', languages: ['en', 'ro'], soft_skills: ['Creativity'], open_to_work: true}),
  (u5:User {id: 'u5', name: 'Elena Radu', role: 'Junior Java Dev', languages: ['ro'], soft_skills: ['Teamwork'], open_to_work: false}),
  (u6:User {id: 'u6', name: 'Florin Dumitrescu', role: 'Fullstack Dev', languages: ['en'], soft_skills: ['Adaptability'], open_to_work: true}),
  (u7:User {id: 'u7', name: 'George Vasilescu', role: 'Product Manager', languages: ['en', 'es'], soft_skills: ['Negotiation'], open_to_work: false}),
  (u8:User {id: 'u8', name: 'Ioana Marin', role: 'HR Recruiter', languages: ['en', 'ro'], soft_skills: ['Communication'], open_to_work: false}),
  (u9:User {id: 'u9', name: 'Matei Pop', role: 'Student', languages: ['en'], soft_skills: ['Learning'], open_to_work: true}),
  (u10:User {id: 'u10', name: 'Nina Voicu', role: 'Financial Analyst', languages: ['en', 'fr'], soft_skills: ['Analysis'], open_to_work: true}),
  (u11:User {id: 'u11', name: 'Paul Serban', role: 'Developer Advocate', languages: ['en', 'ro', 'de'], soft_skills: ['Public Speaking'], open_to_work: false}),
  (u12:User {id: 'u12', name: 'Raluca Nistor', role: 'CTO', languages: ['en'], soft_skills: ['Strategy'], open_to_work: false})
"""
run_query(useri)
print("Utilizatori creați!")

Utilizatori creați!


### Relatii Noduri

In [407]:
relNoduri = """
MATCH (u1:User {id:'u1'}), (u2:User {id:'u2'}), (u3:User {id:'u3'}),
      (u4:User {id:'u4'}), (u5:User {id:'u5'}), (u6:User {id:'u6'}),
      (u7:User {id:'u7'}), (u8:User {id:'u8'}), (u9:User {id:'u9'}),
      (u10:User {id:'u10'}), (u11:User {id:'u11'}), (u12:User {id:'u12'})

MATCH (c_google:Company {name:'Google'}), (c_uipath:Company {name:'UiPath'}),
      (c_bitdef:Company {name:'Bitdefender'}), (c_bcr:Company {name:'BCR'}),
      (c_rev:Company {name:'Revolut'})

MATCH (uni_poli:University {name:'Politehnica Bucuresti'}), (uni_unibuc:University {name:'Universitatea Bucuresti'}),
      (uni_ubb:University {name:'UBB Cluj'}), (uni_ase:University {name:'ASE'})

MATCH (cert_aws:Certification {name:'AWS Solutions Architect'}), (cert_cka:Certification {name:'CKA (Kubernetes)'})

MATCH (proj_ai:Project {name:'OpenVision AI'}), (proj_edu:Project {name:'LearnGraph'})

MATCH (post_tech:Post {id:'p001'})

CREATE
  // STUDIED_AT
  (u1)-[:STUDIED_AT {degree: 'Master', year: 2015}]->(uni_poli),
  (u2)-[:STUDIED_AT {degree: 'Master', year: 2015}]->(uni_poli),
  (u5)-[:STUDIED_AT {degree: 'Bachelor', year: 2023}]->(uni_unibuc),
  (u9)-[:STUDIED_AT {degree: 'Bachelor', year: 2024}]->(uni_unibuc),
  (u3)-[:STUDIED_AT {degree: 'Bachelor', year: 2018}]->(uni_ubb),
  (u4)-[:STUDIED_AT {degree: 'Bachelor', year: 2020}]->(uni_ubb),
  (u7)-[:STUDIED_AT]->(uni_ase),
  (u10)-[:STUDIED_AT]->(uni_ase),

  // WORKS_AT
  (u1)-[:WORKS_AT {role: 'Lead', current: true}]->(c_google),
  (u11)-[:WORKS_AT {role: 'Advocate', current: true}]->(c_google),
  (u2)-[:WORKS_AT {role: 'DevOps', current: true}]->(c_uipath),
  (u6)-[:WORKS_AT {role: 'Dev', current: true}]->(c_uipath),
  (u3)-[:WORKS_AT {role: 'SecEng', current: true}]->(c_bitdef),
  (u10)-[:WORKS_AT {role: 'Analyst', current: true}]->(c_bcr),
  (u7)-[:WORKS_AT {role: 'PO', current: false, end_year: 2022}]->(c_bcr),
  (u7)-[:WORKS_AT {role: 'Senior PM', current: true}]->(c_rev),

  // Certificări & Proiecte
  (u1)-[:EARNED {year: 2022}]->(cert_aws),
  (u2)-[:EARNED {year: 2023}]->(cert_aws),
  (u3)-[:EARNED {year: 2021}]->(cert_cka),
  (u4)-[:CONTRIBUTED_TO {commits: 150}]->(proj_ai),
  (u6)-[:CONTRIBUTED_TO {commits: 80}]->(proj_ai),
  (u9)-[:CONTRIBUTED_TO {commits: 5}]->(proj_edu),
  (u11)-[:CONTRIBUTED_TO {commits: 500}]->(proj_edu),

  // FOLLOWS & Content
  (u1)-[:FOLLOWS]->(u2),
  (u2)-[:FOLLOWS]->(u3),
  (u8)-[:FOLLOWS]->(u2),
  (u12)-[:FOLLOWS]->(u1),
  (u7)-[:FOLLOWS]->(u10),
  (u4)-[:FOLLOWS]->(u11),

  (u11)-[:AUTHORED]->(post_tech),
  (u1)-[:ENGAGED_WITH {type: 'Like'}]->(post_tech),
  (u9)-[:ENGAGED_WITH {type: 'Comment'}]->(post_tech)
"""
run_query(relNoduri)
print("Relații create!")

Relații create!


## Graph Traversal Queries

- Shortest Path între doi utilizatori (pe relația FOLLOWS)
- Friends-of-Friends (rețea socială la 2 niveluri)
- Recommended Contacts (bazat pe conexiuni comune)
- Similar Users (pe baza studiilor și locurilor de muncă)
- Content Recommendations (postări relevante din rețea)


In [408]:
# 1) SHORTEST PATH
print("Shortest Path între u12 și u1: ")

query_shortest = """
MATCH (a:User {id:$start}), (b:User {id:$end})
MATCH path = shortestPath( (a)-[:FOLLOWS*..6]->(b) )
RETURN path
"""

print(run_query(query_shortest, {"start": "u12", "end": "u1"}))



Shortest Path între u12 și u1: 
[{'path': [{'role': 'CTO', 'soft_skills': ['Strategy'], 'languages': ['en'], 'open_to_work': False, 'name': 'Raluca Nistor', 'id': 'u12'}, 'FOLLOWS', {'role': 'Staff Engineer', 'soft_skills': ['Leadership'], 'languages': ['en', 'ro'], 'open_to_work': False, 'name': 'Ana Popa', 'id': 'u1'}]}]


In [409]:
# 2) FRIENDS OF FRIENDS (FOLLOWS)
print("\n Friends-of-Friends pentru u1")

query_fof = """
MATCH (u:User {id:$id})-[:FOLLOWS]->(friend)-[:FOLLOWS]->(fof)
WHERE fof.id <> $id AND NOT (u)-[:FOLLOWS]->(fof)
RETURN DISTINCT fof
"""

print(run_query(query_fof, {"id": "u1"}))




 Friends-of-Friends pentru u1
[{'fof': {'role': 'Security Expert', 'soft_skills': ['Audit'], 'languages': ['en', 'de'], 'open_to_work': False, 'name': 'Cristi Stan', 'id': 'u3'}}]


In [410]:
# 3) RECOMMENDED CONTACTS (mutual follows)
print("\nRecommended Contacts pentru u1")

query_reco = """
MATCH (me:User {id:$id})-[:FOLLOWS]->(x)-[:FOLLOWS]->(rec)
WHERE rec.id <> $id AND NOT (me)-[:FOLLOWS]->(rec)
RETURN rec, COUNT(x) AS mutual
ORDER BY mutual DESC
"""

print(run_query(query_reco, {"id": "u1"}))



Recommended Contacts pentru u1
[{'rec': {'role': 'Security Expert', 'soft_skills': ['Audit'], 'languages': ['en', 'de'], 'open_to_work': False, 'name': 'Cristi Stan', 'id': 'u3'}, 'mutual': 1}]


In [411]:
# 4) SIMILAR USERS (based on studies + workplaces)
print("\nSimilar Users pentru u3 (education + work)")

query_similar = """
MATCH (me:User {id:$id})-[:STUDIED_AT|WORKS_AT]->(n)<-[:STUDIED_AT|WORKS_AT]-(other)
WHERE me.id <> other.id
RETURN other, COUNT(n) AS score
ORDER BY score DESC
"""

print(run_query(query_similar, {"id": "u3"}))



Similar Users pentru u3 (education + work)
[{'other': {'role': 'Frontend Dev', 'soft_skills': ['Creativity'], 'languages': ['en', 'ro'], 'open_to_work': True, 'name': 'David Munteanu', 'id': 'u4'}, 'score': 1}]


In [412]:
# 5) CONTENT RECOMMENDATION (authors you follow)
print("\nContent Recommendations pentru u4")

query_content = """
MATCH (me:User {id:$id})-[:FOLLOWS]->(author)-[:AUTHORED]->(post)
RETURN post.id AS postId, author.name AS author
"""

print(run_query(query_content, {"id": "u4"}))



Content Recommendations pentru u4
[{'postId': 'p001', 'author': 'Paul Serban'}]


## Pattern matching

### Interogare 1 - Selectare aplicanti post
- O interogare care caută angajați "open to work", care au mai lucrat anteriori intr-un domeniu, care poseda anumite soft-skill-uri si care nu ai mai lucrat pentu compania care face recrutarea.
- Pentru aceasta interogare, am ales ca exemple:
    - domenii : Tech, Automation
    - soft-skill-uri: "Leadership" si "Problem Solving"
    - compania care recruteaza: Google
- Am ales DISTINCT pentru past_company.name pentru a nu avea duplicate in cazul in care, in trecut, un candidat a lucrat de mai multe ori la acceasi companie, in perioade diferite

In [413]:
query1 = """MATCH (candidate:User {open_to_work: true})
MATCH (candidate)-[:WORKS_AT]->(past_company:Company)
WHERE past_company.industry IN ['Tech', 'Automation']
  AND ANY(skill IN candidate.soft_skills WHERE skill IN ['Leadership', 'Problem Solving', 'Adaptability'])
  AND NOT (candidate)-[:WORKS_AT {current: true}]->(:Company {name: 'Google'})
WITH candidate, collect(DISTINCT past_company.name) AS experience
RETURN candidate.name AS Nume_Candidat,
       candidate.role AS Rol_Curent,
       experience AS Istoric_Companii,
       candidate.soft_skills AS Abilitati_Soft,
       size(experience) AS Numar_Joburi_Anterioare
ORDER BY Numar_Joburi_Anterioare DESC"""
rezultat = run_query(query1)
df = pd.DataFrame(rezultat)
print(df)

       Nume_Candidat     Rol_Curent Istoric_Companii     Abilitati_Soft  \
0     Bogdan Ionescu    DevOps Lead         [UiPath]  [Problem Solving]   
1  Florin Dumitrescu  Fullstack Dev         [UiPath]     [Adaptability]   

   Numar_Joburi_Anterioare  
0                        1  
1                        1  


### Interogarea 2 - Mathcing Intership

- O interogare care identifica perechi potențiale de Instructor-Junior pentru realizarea stagiului de intership. Se cauta un utilizator "Senior" care are o certificare specifică și un utilizator "Junior" sau "Student" care este interesat de acel domeniu (a interacționat cu o postare pe acel subiect), dar nu sunt încă conectați.
- Pentru acest exemplu am selectat:
    - Certificate: AWS, Kubernetes
    - Topicul postarilor cu care a intercationat posibilul intern: 'AI', 'Cloud', 'DevOps'

In [414]:
query2 = """MATCH (mentor:User)-[:EARNED]->(cert:Certification)
MATCH (mentee:User)-[:ENGAGED_WITH]->(post:Post)
WHERE mentor.id <> mentee.id
  AND NOT (mentee)-[:FOLLOWS]->(mentor)
  AND (
      cert.name CONTAINS 'AWS' OR
      cert.name CONTAINS 'Kubernetes'
  )
  AND ANY(topic IN post.topics WHERE topic IN ['AI', 'Cloud', 'DevOps'])
  AND mentee.role IN ['Student', 'Junior Java Dev']

RETURN mentor.name AS Potential_Mentor,
       cert.name AS Expertise,
       mentee.name AS Potential_Intern,
       post.topics AS Interest_Area"""
rezultat = run_query(query2)
df = pd.DataFrame(rezultat)
print(df)

  Potential_Mentor                Expertise Potential_Intern Interest_Area
0         Ana Popa  AWS Solutions Architect        Matei Pop  [AI, Future]
1   Bogdan Ionescu  AWS Solutions Architect        Matei Pop  [AI, Future]
2      Cristi Stan         CKA (Kubernetes)        Matei Pop  [AI, Future]


### Interogarea 3 - Detectarea Potențialului de Co-autorat

- Interogarea cauta doi utilizatori care nu au lucrat împreună și nu au contribuit la același proiect, dar au o bază comună de comunicare (limbi străine) care ar facilita colaborarea, deși lucrează pe proiecte tehnice distincte."

In [415]:
query3 = """
MATCH (u1:User)-[:CONTRIBUTED_TO]->(p1:Project)
MATCH (u2:User)-[:CONTRIBUTED_TO]->(p2:Project)
WHERE p1.name <> p2.name
  AND u1.id < u2.id
WITH u1, u2, p1, p2,
     [l IN u1.languages WHERE l IN u2.languages] AS limbi_comune

WHERE size(limbi_comune) > 0

RETURN u1.name AS User_1,
       p1.name AS Proiect_1,
       u2.name AS User_2,
       p2.name AS Proiect_2,
       limbi_comune AS Limbi_Comune
ORDER BY size(limbi_comune) DESC
"""
rezultat= pd.DataFrame(run_query(query3))
print(rezultat)

              User_1      Proiect_1             User_2      Proiect_2  \
0        Paul Serban     LearnGraph     David Munteanu  OpenVision AI   
1        Paul Serban     LearnGraph  Florin Dumitrescu  OpenVision AI   
2     David Munteanu  OpenVision AI          Matei Pop     LearnGraph   
3  Florin Dumitrescu  OpenVision AI          Matei Pop     LearnGraph   

  Limbi_Comune  
0     [en, ro]  
1         [en]  
2         [en]  
3         [en]  


### Interogare 4 - Analiza "Brain Drain"

- Aceasta interogare analizează mișcarea angajaților pentru a vedea dacă există un pattern de migrație de la o companie la alta

In [416]:
query4 = """
MATCH (u:User)-[r1:WORKS_AT {current: false}]->(old_comp:Company)
MATCH (u)-[r2:WORKS_AT {current: true}]->(new_comp:Company)
WHERE old_comp.name <> new_comp.name
RETURN old_comp.name AS Companie_Veche,
       old_comp.work_culture AS Cultura_Veche,
       new_comp.name AS Companie_Noua,
       new_comp.work_culture AS Cultura_Noua,
       collect(u.name) AS Angajati_Migrati,
       count(u) AS Volum_Migratie
ORDER BY Volum_Migratie DESC
"""
rezultat = run_query(query4)
df = pd.DataFrame(rezultat)
print(df)

  Companie_Veche        Cultura_Veche Companie_Noua    Cultura_Noua  \
0            BCR  [Corporate, Stable]       Revolut  [Hustle, Data]   

     Angajati_Migrati  Volum_Migratie  
0  [George Vasilescu]               1  


### Interogarea 5 - Recomandare Socială

- Găsește utilizatori care ar trebui să se cunoască deoarece au prieteni comuni si au absolvit aceeași universitate

In [417]:
query5 = """
MATCH (u1:User)-[r1:STUDIED_AT]->(uni:University)<-[r2:STUDIED_AT]-(u2:User)
WHERE u1.id < u2.id
  AND NOT (u1)-[:FOLLOWS]-(u2)

RETURN u1.name AS User_1,
       u2.name AS User_2,
       uni.name AS Universitate_Comuna,
       coalesce(abs(r1.year - r2.year), 0) AS Diferenta_Ani_Absolvire
ORDER BY Universitate_Comuna
"""
df5 = pd.DataFrame(run_query(query5))
print(df5)

        User_1          User_2      Universitate_Comuna  \
0  Cristi Stan  David Munteanu                 UBB Cluj   
1   Elena Radu       Matei Pop  Universitatea Bucuresti   

   Diferenta_Ani_Absolvire  
0                        2  
1                        1  


### Interogarea 6 - Echipa Hackathon

- Aceasta interogare cauta 3 persoane pentru formarea unei echipe pentru un Heckathlon: o persoana pe frontend sau fullstack, un leader, cu anumite soft-skill-uri si o persoana pe backend (python).
- Echipa trebuie sa fie in acelasi oras, prin intermediul companiei la care lucreaza, sau job-ul sa fie remote

In [418]:
query6 = """
MATCH (dev:User)
WHERE dev.role CONTAINS 'Dev' OR dev.role CONTAINS 'Frontend'

MATCH (lead:User)
WHERE ANY(s IN lead.soft_skills WHERE s IN ['Leadership', 'Management', 'Strategy'])
  AND lead.id <> dev.id
MATCH (expert:User)-[:CONTRIBUTED_TO]->(p:Project)
WHERE 'Python' IN p.tech_stack
  AND expert.id <> dev.id AND expert.id <> lead.id

MATCH (dev)-[:WORKS_AT {current:true}]->(c1:Company),
      (lead)-[:WORKS_AT {current:true}]->(c2:Company),
      (expert)-[:WORKS_AT {current:true}]->(c3:Company)
WHERE (c1.city = c2.city AND c2.city = c3.city) OR c1.city = 'Remote'

RETURN dev.name AS Developer,
       lead.name AS Leader,
       expert.name AS Python_Expert,
       c1.city AS Location_Base
LIMIT 5
"""
rezultat = pd.DataFrame(run_query(query6))
print(rezultat)

        Developer    Leader      Python_Expert Location_Base
0  Bogdan Ionescu  Ana Popa  Florin Dumitrescu     Bucharest
1     Paul Serban  Ana Popa  Florin Dumitrescu     Bucharest


### Interogare 7 - Influenceri de Conținut

- Interogarea identifică utilizatorii ale căror postări au impact dincolo de rețeaua lor directă

In [419]:
query7= """
MATCH (author:User)-[:AUTHORED]->(post:Post)
MATCH (fan:User)-[r:ENGAGED_WITH]->(post)
WHERE NOT (fan)-[:FOLLOWS]->(author)
  AND fan.id <> author.id

RETURN author.name AS Influencer,
       post.topics AS Topics,
       count(fan) AS Viral_Reach,
       collect(DISTINCT r.type) AS Engagement_Types
ORDER BY Viral_Reach DESC
"""
rezultat = pd.DataFrame(run_query(query7))
print(rezultat)

    Influencer        Topics  Viral_Reach Engagement_Types
0  Paul Serban  [AI, Future]            2  [Like, Comment]


## Inchidere Conexiune

In [420]:
driver.close()